In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests

from bs4 import BeautifulSoup
import QuantLib as ql

In [3]:
from quant_lib import curve as cv

## Calculating Bond Price, Duration, Convexity

##### $ V = Pe^{-y(T-t)} + \sum_{i=1}^{T} C_i e^{-y(t_i - t)} $

where, $V$ is bond price, $P$ is notional amount, $y$ is market interest rate, $N$ is number of coupon payment, $C_i$ is $i_{th}$ coupon payment amount 

##### $ Duration = -\frac{1}{V(y)} \frac{dy}{dV}$
this is modifed Duration (different from Macaulay Duration)   

##### $ Convexity = \frac{1}{V(y)} \frac{d y^2}{d^2 V}$

In [4]:
ref_date = cv.get_date()
quote = cv.get_quote(ref_date)
print(quote)
curve = cv.treasury_curve(ref_date, quote)
print(curve)

             days  price coupon
maturity                       
2022-11-08     33  2.975    0.0
2023-01-05     91  3.263    0.0
2023-04-06    182  4.043    0.0
2023-10-05    364  4.216    0.0
2024-09-30    725  4.283   4.25
2025-09-15   1075  4.307    3.5
2027-09-30   1820  4.104  4.125
2029-09-30   2551  3.995  3.875
2032-08-15   3601  3.848   2.75
2052-08-15  10906  3.798    3.0
<QuantLib.QuantLib.PiecewiseLinearZero; proxy of <Swig Object of type 'ext::shared_ptr< PiecewiseLinearZero > *' at 0x7f94bceefdb0> >


In [5]:
# convert into Engine
spotCurveHandle = ql.YieldTermStructureHandle(curve)
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)

In [9]:
# Treasury Bond Specifiction
issueDate = ql.Date(2,10,2022)
maturityDate = ql.Date(2,10,2032)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates()
convention = ql.ModifiedFollowing
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule(
                issueDate,
                maturityDate,
                tenor,
                calendar,
                convention,
                convention,
                dateGeneration,
                monthEnd
                    )
dayCount = ql.ActualActual()
couponRate = [0.02]
settlementDays = 1
faceValue = 100

In [10]:
# set Fixed Rate bond pricing engine
fixedRateBond = ql.FixedRateBond(
    settlementDays,
    faceValue,
    schedule,
    couponRate,
    dayCount
)

# Conduct bond pricing
fixedRateBond.setPricingEngine(bondEngine)

In [11]:
# calculate YTM
targetPrice = fixedRateBond.cleanPrice()
ytm = ql.InterestRate(
    fixedRateBond.bondYield(targetPrice, dayCount, ql.Compounded, ql.Semiannual),
    dayCount,
    ql.Compounded,
    ql.Semiannual
)

In [15]:
print("Yield to maturity = {:.4f}".format(ytm.rate()))
print("Duration = {:.4f}".format(ql.BondFunctions.duration(fixedRateBond, ytm)))
print("Duration = {:.4f}".format(ql.BondFunctions.convexity(fixedRateBond, ytm)))


Yield to maturity = 0.0272
Duration = 8.9461
Duration = 89.5908
